# 3. 벡터와 행렬 연산 (II)

## 3.1. 행렬-행렬 곱

1. 순차코드

    <img src = "images/image08.png">

In [2]:
import numpy as np

A = np.load("examples/A.npy")
B = np.load("examples/B.npy")

C = np.matmul(A, B)
print (C)


[[2.43808921 1.76992881 1.97417806 2.71006386 2.08325036 1.6900913
  2.2726759  0.84911753 2.24487038 2.89619951]
 [3.32436378 2.26264606 2.84394628 3.65224577 2.81632642 2.83825689
  2.82821487 1.53278552 2.67656844 4.07527853]
 [2.52516491 1.63464914 2.40555223 2.65467965 1.77237814 2.67850384
  2.31310182 1.51563888 2.0609549  3.06054468]
 [2.09033953 1.76774597 1.67791452 2.11438343 2.12795604 2.40671103
  1.95276353 1.62493618 0.99662127 2.75977504]
 [1.53354765 1.30140395 1.89016566 2.10327574 1.99955199 2.01952688
  1.15680931 1.51912599 1.22258369 2.13703296]
 [3.72882467 2.41779476 3.16969681 3.68080586 2.81891888 3.64870978
  3.12966289 2.03779475 2.55999561 4.32804162]
 [2.5376105  2.0037945  2.32930118 2.77907609 2.72584889 2.90360689
  1.73261574 2.18891316 1.53259861 3.31508807]
 [1.60960489 1.2524839  1.58606037 1.98764014 1.34049028 1.59212055
  1.49071873 0.90336752 1.38118709 2.23947988]
 [1.79226644 1.576982   2.25868466 2.59114148 2.65784558 2.53265303
  1.22004361 

2. 행렬A의 행분할

    <img src = "images/image09.png">

In [3]:
%%writefile examples/AB.py

import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD

rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0 :
    A = np.load("examples/A.npy")
    B = np.load("examples/B.npy")
    n = A[0].size
    n = comm.bcast(n, root = 0)

else :
    n = 0
    n = comm.bcast(n, root = 0)
    A = None
    B = np.empty((n, n), dtype = np.float64)

n_row = int(n / size)

A_row = np.empty((n_row, n), dtype = np.float64)

##### 행렬 A의 분할
comm.Scatter(A, A_row, root = 0) #FIX ME
comm.Bcast(B, root = 0)

C = np.matmul(A_row,B)

print(C, rank)

Writing examples/AB.py


In [4]:
! mpiexec -np 2 python examples/AB.py

[[2.43808921 1.76992881 1.97417806 2.71006386 2.08325036 1.6900913
  2.2726759  0.84911753 2.24487038 2.89619951]
 [3.32436378 2.26264606 2.84394628 3.65224577 2.81632642 2.83825689
  2.82821487 1.53278552 2.67656844 4.07527853]
 [2.52516491 1.63464914 2.40555223 2.65467965 1.77237814 2.67850384
  2.31310182 1.51563888 2.0609549  3.06054468]
 [2.09033953 1.76774597 1.67791452 2.11438343 2.12795604 2.40671103
  1.95276353 1.62493618 0.99662127 2.75977504]
 [1.53354765 1.30140395 1.89016566 2.10327574 1.99955199 2.01952688
  1.15680931 1.51912599 1.22258369 2.13703296]] 0
[[3.72882467 2.41779476 3.16969681 3.68080586 2.81891888 3.64870978
  3.12966289 2.03779475 2.55999561 4.32804162]
 [2.5376105  2.0037945  2.32930118 2.77907609 2.72584889 2.90360689
  1.73261574 2.18891316 1.53259861 3.31508807]
 [1.60960489 1.2524839  1.58606037 1.98764014 1.34049028 1.59212055
  1.49071873 0.90336752 1.38118709 2.23947988]
 [1.79226644 1.576982   2.25868466 2.59114148 2.65784558 2.53265303
  1.220043

3. 행렬A의 행분할 (비등분할)

    <img src = "images/image10.png">

In [5]:
%%writefile examples/AvarB.py

# Matrix A의 Row decomposition

import numpy as np
from mpi4py import MPI
from tools import para_range

comm = MPI.COMM_WORLD

rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0 :
    A = np.load("examples/A.npy")
    B = np.load("examples/B.npy")
    n = A[0].size
    n = comm.bcast(n, root = 0)
else :
    n = 0
    n = comm.bcast(n, root = 0)
    A = None
    B = np.empty((n, n), dtype = np.float64)

##### 행렬 A의 분할 범위
ista, iend = para_range(n, size, rank) #FIX ME

n_row = (iend - ista + 1) #FIX ME

A_row = np.empty((n_row, n), dtype = np.float64)
n_list = comm.gather(n_row * n, root = 0)

##### 행렬 A의 분할
comm.Scatterv([A, n_list], A_row, root = 0) #FIX ME
comm.Bcast(B, root = 0)

C_rows = np.matmul(A_row,B)

print(C_rows, rank)

Writing examples/AvarB.py


In [6]:
! mpiexec -np 3 python examples/AvarB.py

[[1.53354765 1.30140395 1.89016566 2.10327574 1.99955199 2.01952688
  1.15680931 1.51912599 1.22258369 2.13703296]
 [3.72882467 2.41779476 3.16969681 3.68080586 2.81891888 3.64870978
  3.12966289 2.03779475 2.55999561 4.32804162]
 [2.5376105  2.0037945  2.32930118 2.77907609 2.72584889 2.90360689
  1.73261574 2.18891316 1.53259861 3.31508807]] [[1.60960489 1.2524839  1.58606037 1.98764014 1.34049028 1.59212055
  1.49071873 0.90336752 1.38118709 2.23947988]
 [1.79226644 1.576982   2.25868466 2.59114148 2.65784558 2.53265303
  1.22004361 2.00448121 1.39171133 2.70871931]
 [2.51597362 2.15609642 2.40916312 2.89465763 2.40953836 2.29254105
  2.20112086 1.70694314 1.93419173 3.24786593]] 2
1
[[2.43808921 1.76992881 1.97417806 2.71006386 2.08325036 1.6900913
  2.2726759  0.84911753 2.24487038 2.89619951]
 [3.32436378 2.26264606 2.84394628 3.65224577 2.81632642 2.83825689
  2.82821487 1.53278552 2.67656844 4.07527853]
 [2.52516491 1.63464914 2.40555223 2.65467965 1.77237814 2.67850384
  2.313

4. 행렬A의 행분할, 행렬 B의 열분할
  - 각 프로세스가 가진 부분행렬만 곱할 경우

    <img src = "images/image11_1.png">

  - 통신을 이용하여 $B_i$에 대한 부분행렬을 수신하여 부분행렬 곱을 실행

    <img src = "images/image11_2.png">


In [7]:
%%writefile examples/ABvar.py

import numpy as np
from mpi4py import MPI
from tools import para_range

comm = MPI.COMM_WORLD

rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0 :
    A = np.load("examples/A.npy")
    B = np.load("examples/B.npy")
    BT = np.transpose(B).copy()
    n = A[0].size
    n = comm.bcast(n, root = 0)

else :
    n = 0
    n = comm.bcast(n, root = 0)
    A = None
    BT = None

ista, iend = para_range(n, size, rank)

n_row = (iend - ista + 1)

A_row = np.empty((n_row, n), dtype = np.float64)
BT_row = np.empty((n_row, n), dtype = np.float64)
n_rows = comm.allgather(n_row * n)

##### 행렬 A, B의 분할
comm.Scatterv([A, n_rows], A_row, root = 0) #FIX ME
comm.Scatterv([BT, n_rows], BT_row, root = 0) #FIX ME
B_col = np.transpose(BT_row)

inext = rank + 1 if rank < size - 1 else 0
iprev = rank - 1 if rank > 0 else size - 1

C_unordered_rows = np.matmul(A_row, B_col)

for i in range(size - 1) :
    iloc = iprev - i if iprev >= i else iprev - i + size
    B_recv = np.empty(n_rows[iloc], dtype = np.float64)

    ##### 분할된 행렬 B를 송수신하고 A의 분할된 부분과 곱하여 C에 저장
    comm.Sendrecv(B_col, inext, 1, B_recv, iprev, 1) #FIX ME
    B_col = np.copy(B_recv)
    B_col = np.reshape(B_col, (n, int(n_rows[iloc]/n))) #FIX ME
    C_block = np.matmul(A_row, B_col) #FIX ME
    C_unordered_rows = np.append(C_unordered_rows, C_block, axis = 1) #FIX ME

print(C_unordered_rows, rank)

Writing examples/ABvar.py


In [8]:
! mpiexec -np 3 python examples/ABvar.py

[[0.90336752 1.38118709 2.23947988 1.35979991 2.13047487 1.51525464
  1.28586525 0.92749032 1.55481306 1.63314528]
 [2.00448121 1.39171133 2.70871931 1.7708386  2.31727939 1.65989538
  2.47912148 1.74728039 2.628282   2.34119267]
 [1.70694314 1.93419173 3.24786593 2.13460666 2.77527543 2.7274958
  2.1397937  1.37625043 2.45464219 2.83655652]] 2
[[2.43808921 1.76992881 1.97417806 2.71006386 2.49735365 1.65893238
  1.7758255  2.05296055 2.85343892 2.52931704]
 [3.32436378 2.26264606 2.84394628 3.65224577 3.2008964  2.61809047
  2.79110772 2.57504371 3.82863666 2.99459153]
 [2.52516491 1.63464914 2.40555223 2.65467965 2.36573203 1.29833334
  2.04687863 2.09761675 2.3512836  3.02689083]
 [2.09033953 1.76774597 1.67791452 2.11438343 1.9122912  2.09330263
  1.67607002 1.85661096 1.96529024 1.98148748]] 0
[[1.99955199 2.01952688 1.15680931 1.94381772 1.41273001 2.04580829
  1.92631513 1.87876781 1.44841558 2.03937034]
 [2.81891888 3.64870978 3.12966289 3.1425895  2.36141592 3.33864165
  3.701

5. 직접 해보기
    - 올바른 크기의 C를 미리 선언하고 적절한 위치에 C_block 을 배치
    - 3의 C_rows와 같은 결과를 얻음
